In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as Func
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Criar uma SparkSession
spark = SparkSession.builder.appName("Exemplo de Dataframe").getOrCreate()

In [ ]:
#criar um data frame simples, sem schema
df1 = spark.createDataFrame([
    ("Pedro", 10),
    ("Maria",20),
    ("José",40)
])

#show é ação, então tudo o que foi feito anteriormente é executado, lazzy
df1.show()

In [ ]:
#criar df com schema
schema = "Id INT, Nome STRING"
dados = [
    (1, "Pedro"),
    (2, "Maria"),
    (3, "José")
]

df2 = spark.createDataFrame(dados, schema)
df2.show()

In [ ]:
#com transformação
schema2 = "Produtos STRING, Vendas INT"
vendas = [
    ["Caneta", 10],
    ["Lápis", 20],
    ["Caneta", 40]
]

df3 = spark.createDataFrame(vendas , schema2)

#podemos contatenar as operações, neste caso sem persitir
df3.groupBy("Produtos").agg(sum("Vendas")).show()

In [ ]:
agrupado = df3.groupBy("Produtos").agg(sum("Vendas"))
agrupado.show()

In [ ]:
#selecionar colunas específicas
df3.select("Produtos").show()

In [ ]:
df3.select("Produtos", "Vendas").show()

In [ ]:
#expressões e select
df3.select("Produtos", "Vendas", expr("Vendas * 0.2")).show()

In [ ]:
#para ver o schema
df3.schema

In [ ]:
#ver colunas
df3.columns

In [ ]:
# Define o esquema do arquivo CSV com os tipos de dados para cada coluna
schema_arq = "id INT, " \
    "nome STRING, " \
    "status STRING, " \
    "cidade STRING, " \
    "vendas INT, " \
    "data STRING"

# Define o caminho do arquivo CSV que será lido
despachantes_csv = "../../arquivos/download/despachantes.csv"

# Lê o arquivo CSV usando o Spark, sem cabeçalho e aplicando o esquema definido
despachantes = spark.read.csv(
    despachantes_csv, 
    header = False, 
    schema = schema_arq
)

# Exibe as primeiras linhas do DataFrame
despachantes.show()

# Exibe o esquema do DataFrame
despachantes.schema

In [ ]:
#condição lógica com where
despachantes.select("id", "nome", "vendas").where(Func.col("vendas") > 20).show()

In [ ]:
#& para and, | para or, e ~ para not
despachantes.select("id","nome","vendas").where((Func.col("vendas") > 20) & (Func.col("vendas") < 40)).show()

In [ ]:
#renomear coluna
novodf = despachantes.withColumnRenamed("nome","nomes")
novodf.columns

In [ ]:
#coluna data está como string, vamos transformar em texto
despachantes2 = despachantes.withColumn("data2", to_timestamp(Func.col("data"), "yyyy-MM-dd"))
despachantes2.schema

In [ ]:
#operações sobre datas
despachantes2.select(year("data")).show()
despachantes2.select(year("data")).distinct().show()
despachantes2.select("nome",year("data")).orderBy("nome").show()
despachantes2.select("data").groupBy(year("data")).count().show()
despachantes2.select(Func.sum("vendas")).show()

In [ ]:
# Para o Spark no final do script, se ainda não foi parado
if spark.sparkContext._jsc.sc().isStopped() == False:
    spark.stop()